# %run init_notebookspace.py

In [ ]:
import pandas as pd
import numpy as np

from pandas_datareader import fred
import yfinance as yf
import pandas_datareader as pdread

import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from yahoo_fin import stock_info as ysi

from itertools import chain

In [4]:
from src.utils import apply_datetime_format

NameError: name 'ysi' is not defined

In [ ]:
# get sp500 composits & market cap
sp500_tickers = ysi.tickers_sp500()
df = pd.DataFrame(index=sp500_tickers, 
                  columns=['market_cap'], 
                  data=[pdread.get_quote_yahoo(item)['marketCap'].values[0] for item in sp500_tickers])
sp500_largest = df.sort_values('market_cap', ascending=False).index[:5].values
sp500_ticker = ['^GSPC']

In [ ]:
start = '2020-01-01'
end = '2022-12-31'

In [ ]:
df_prices = pd.DataFrame()
df_prices.index = pd.date_range(start, periods=(apply_datetime_format(end, '%Y-%m-%d') - apply_datetime_format(start, '%Y-%m-%d')).days)

for item in [*chain(sp500_largest, sp500_ticker)]:
    data = yf.download(item, start, end)
    data.columns = list([f'{item}_{x}' for x in data.columns])
    df_prices = df_prices.join(data)
    
# get closing price
df_c = df_prices[[item for item in df_prices.columns if 'Adj Close' in item]].copy()
df_c.columns = [item[:-10] for item in df_c.columns]
df_c.dropna(inplace=True)

In [ ]:
# get log returns
df_rets = np.log(df_c / df_c.shift(1)).dropna()
df_rets.sort_index(inplace=True)
df_rets.asfreq = "D"

df_rets['^GSPC_lead'] = df_rets['^GSPC'].shift(1)

df_rets.dropna(inplace=True)
df_rets.index.asfreq = 'd'


In [ ]:
# save data to file
df_rets.to_csv(os.path.join(DATA_DIR, 'returns.csv'))

In [ ]:
# load data from file
df_rets = pd.read_csv(os.path.join(DATA_DIR, 'returns.csv'))

In [ ]:
# Checking for statoinarity via ADF test below
res = adfuller( returns.csv.value.dropna())  
print('Augmented Dickey-Fuller Statistic: %f' % res[0])  
print('p-value: %f' % res[1])

In [ ]:
# determining the ADF for the 'd' value of ARIMA 


In [ ]:
# Next looking at autocorrelation 
import numpy as np, pandas as pd  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  
import matplotlib.pyplot as plt 
plt.rcParams.update({'figure.figsize' : (9,7), 'figure.dpi' : 120}) 

In [ ]:
# Importing data  
df = pd.read_csv('returns.csv', names = ['value'], header = 0)  

In [ ]:
# The Genuine Series  
fig, axes = plt.subplots(3, 2, sharex = True)  
axes[0, 0].plot(df.value); axes[0, 0].set_title('The Genuine Series')  
plot_acf(df.value, ax = axes[0, 1])  

In [ ]:
# Order of Differencing: First  
axes[1, 0].plot(df.value.diff()); axes[1, 0].set_title('Order of Differencing: First')  
plot_acf(df.value.diff().dropna(), ax = axes[1, 1])  

In [ ]:
# Order of Differencing: Second  
axes[2, 0].plot(df.value.diff().diff()); axes[2, 0].set_title('Order of Differencing: Second')  
plot_acf(df.value.diff().diff().dropna(), ax = axes[2, 1])  
plt.show()  


In [ ]:
# Determine the differencing above. Weak/ strong stationarity? 
# Next, looking at how to make the data stationary via 3 tests. 

In [ ]:
from pmdarima.arima.utils import ndiffs  
import pandas as pd  

In [ ]:
df = pd.read_csv('returns.csv', names = ['value'], header = 0)  
X = df.value  

In [ ]:
# Augmented Dickey Fuller Test  
adftest = ndiffs(X, test = 'adf')  

In [ ]:
# KPSS Test  
kpsstest = ndiffs(X, test = 'kpss')  

In [ ]:
# PP Test  
pptest = ndiffs(X, test = 'pp')  

In [ ]:
print("ADF Test =", adftest)  
print("KPSS Test =", kpsstest)  
print("PP Test =", pptest) 

In [ ]:
#Next finding the 'p' term for AR via a partial auocorrelatoin graph

In [ ]:
import numpy as np, pandas as pd  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  
import matplotlib.pyplot as plt  

In [ ]:
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})  

In [ ]:
# importing data  
df = pd.read_csv('returns.csv', names = ['value'], header = 0)  

In [ ]:
fig, axes = plt.subplots(1, 2, sharex = True)  
axes[0].plot(df.value.diff()); axes[0].set_title('Order of Differencing: First')  
axes[1].set(ylim = (0,5))  
plot_pacf(df.value.diff().dropna(), ax = axes[1])  
plt.show()  

In [ ]:
# Look a the blue region to determine if it is useful or not

In [ ]:
#Next, finding the 'q' MA 
import numpy as np, pandas as pd  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  
import matplotlib.pyplot as plt  
  
plt.rcParams.update({'figure.figsize' : (9,3), 'figure.dpi' : 120})  

In [ ]:
# importing data  
mydata = pd.read_csv('returns.csv', names = ['value'], header = 0)  

In [ ]:
fig, axes = plt.subplots(1, 2, sharex = True)  
axes[0].plot(mydata.value.diff()); axes[0].set_title('Order of Differencing: First')  
axes[1].set(ylim = (0, 1.2))  
plot_acf(mydata.value.diff().dropna(), ax = axes[1])  
plt.show() 

In [ ]:
# Looking at the autocorrelation graph, we can determine the number needed for q 


In [ ]:
# Now building the ARIMA model 

import numpy as np, pandas as pd  
from statsmodels.tsa.arima_model import ARIMA  

In [ ]:
# importing data  
mydata = pd.read_csv('returns.csv', names = ['value'], header = 0)  

In [ ]:
# Creating ARIMA model  
# change the () to whatever numbers we get from the results for p,d,q above 
mymodel = ARIMA(mydata.value, order = (1, 1, 2))  
modelfit = mymodel.fit(disp = 0)  
print(modelfit.summary())  

In [ ]:
# These are the results. We can alter the output by changing the AIC/ BIC if we have a close call. 

# Now we plot the residuals to ensure that there is no constant mean or varaince to worry about 

In [ ]:
import numpy as np, pandas as pd  
from statsmodels.tsa.arima_model import ARIMA  
import matplotlib.pyplot as plt  

In [ ]:
plt.rcParams.update({'figure.figsize' : (9,3), 'figure.dpi' : 120})  

In [ ]:
# importing data  
mydata = pd.read_csv('returns.csv', names = ['value'], header = 0)  

In [ ]:
# Creating ARIMA model  
mymodel = ARIMA(mydata.value, order = (1, 1, 1))  
modelfit = mymodel.fit(disp = 0)  

In [ ]:
# Plotting Residual Errors  
myresiduals = pd.DataFrame(modelfit.resid)  
fig, ax = plt.subplots(1,2)  
myresiduals.plot(title = "Residuals", ax = ax[0])  
myresiduals.plot(kind = 'kde', title = 'Density', ax = ax[1])  
plt.show()  

In [ ]:
# finally, graphing our predicted versus actual values for trend analysis 

In [ ]:
import numpy as np, pandas as pd  
from statsmodels.tsa.arima_model import ARIMA  
import matplotlib.pyplot as plt  

In [ ]:
plt.rcParams.update({'figure.figsize' : (9,3), 'figure.dpi' : 120})  


In [ ]:
# importing data  
mydata = pd.read_csv('mydataset.csv', names = ['value'], header = 0)  

In [ ]:
# Creating ARIMA model  
mymodel = ARIMA(mydata.value, order = (1, 1, 1))  
modelfit = mymodel.fit(disp = 0)  

# Actual vs Fitted  
modelfit.plot_predict(dynamic = False)  
plt.show()